# Phase 2: Break It

In [3]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

# install checklist
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

#!python3 -m spacy download en_core_web_sm
# install spacy
import spacy

In [4]:
TRAIN = '../data/interim/train.csv'

In [5]:
data_train = pd.read_json('../data/raw/music_reviews_train.json.gz', lines=True)
data_train['sentiment']

0        positive
1        negative
2        negative
3        positive
4        positive
           ...   
99995    positive
99996    negative
99997    positive
99998    negative
99999    negative
Name: sentiment, Length: 100000, dtype: object

In [6]:
labels = data_train['sentiment'].tolist()

In [7]:
input_data = []
label = [] 
c = 0
n = 100 # set how many training examples to use.
with open(TRAIN, mode='r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        if c < n:
            input_data.append(lines[0])
            label.append(int(lines[1]))
            c+=1
        else:
            break

In [21]:
def tkn(sentence):
    """Function to find all tokens in a given sentence
    """
    tok = re.compile('[\'\"]|[A-Za-z]+|[.?!:\'\"]+')
    
    return tok.findall(sentence)
    
def splitsies(para):
    punct = re.compile('[.?!:]')
    t = punct.split(para)
    spl= []
    for i in t:
        temp = tkn(i)
        if len(temp) > 0:
            spl.append(temp)
        
    return spl
    
#splitsies(input_data[0])

In [9]:
print(f'Sentence 1:\n{input_data[0]}\nLabel: {label[0]}')
type(input_data)

Sentence 1:
Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!
Label: 1


list

In [10]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

token_text = []
for para in input_data:
    token_text.append(sent_tokenize(para))
token_text[0]

[nltk_data] Downloading package punkt to /home/dmdequin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Gotta listen to this!',
 'So creative!',
 'Love his music - the words, the message!',
 'Some of my favorite songs on this CD.',
 'I should have bought it years ago!']

In [11]:
nlp = spacy.load('en_core_web_sm')

# Add Typos

In [23]:
#input_data[0], Perturb.add_typos(input_data[0])

In [13]:
ret = Perturb.perturb(input_data, Perturb.add_typos)
ret.data[0]

['Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on this CD. I should have bought it years ago!',
 'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on htis CD. I should have bought it years ago!']

In [14]:
typo = []
for thing in ret.data:
    typo.append(thing[1])
typo[0]

'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on htis CD. I should have bought it years ago!'

In [48]:
#ret = Perturb.perturb(token_text[6], Perturb.add_typos)
#ret.data
for i in range(len(token_text)):
    for j in range(len(token_text[i])):
        if len(token_text[i][j]) <= 1:
            token_text[i][j] = token_text[i][j]+'..'
            
token_text[6]

['Five Stars I love all of his music!', '!..']

In [49]:
p = []
for i in range(len(token_text)):
    ret = Perturb.perturb(token_text[i], Perturb.add_typos)
    typos = []
    for sent in ret.data:
        typos.append(sent[1])
    p.append(typos)

# POS tag data

https://spacy.io/usage/processing-pipelines

When you call nlp on a text, spaCy will tokenize it and then call each component on the Doc, in order. It then returns the processed Doc that you can work with.

In [15]:
pdata = list(nlp.pipe(typo))
#for doc in nlp.pipe(input_data):
    # Do something with the doc here
#    print([(ent.text, ent.label_) for ent in doc.ents])

# Remove End Punctuation

In [16]:
pdata[0], Perturb.strip_punctuation(pdata[0])

(Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on htis CD. I should have bought it years ago!,
 'Gotta listen to this! So creative!  Love his music - the words, the message! Some of my favorite songs on htis CD. I should have bought it years ago')

In [17]:
ret = Perturb.perturb(pdata, Perturb.punctuation)
no_punct = []
for i in ret.data:
    no_punct.append(i[1])
no_punct[0]
len(no_punct)

100

# Negation

In [18]:
pdata = list(nlp.pipe(no_punct))

In [19]:
"""ret = Perturb.perturb(pdata, Perturb.remove_negation)
negated = []
for i in ret.data:
    negated.append(i[1])
negated[0]
len(negated)
pdata[0]"""

'ret = Perturb.perturb(pdata, Perturb.remove_negation)\nnegated = []\nfor i in ret.data:\n    negated.append(i[1])\nnegated[0]\nlen(negated)\npdata[0]'

In [20]:
nega = []
for i in range(100):
    ret = Perturb.remove_negation(pdata[i])
    if ret == None:  # if nothing changes
        nega.append(pdata[i]) # append original sentence
    else: nega.append(ret) # append negated paragraph
len(nega)

# NEED TO CHANGE THE LABELS FOR THE ONES THAT ARE NEGATED

RuntimeError: generator raised StopIteration

# Make Final Output

In [ ]:
output = []
for i in range(len(input_data)):
    dicti = {}
    dicti['reviewText'] = nega[i]
    dicti['sentiment'] = labels[i]
    dicti['category'] = 'category'
    output.append(dicti)
output[0]